# Runtime Test

For conver yolov5 model to trt plan (**yolov5s-2021-07-28.engine**) and build **libmyplugins.so** for your system see instruction: https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5#how-to-run-yolov5s-as-example

Based on this instruction, we wrote a small shell script.

```bash
cd ../inference/convertors/yolo2tensorrt/bin/
./yolov5_tensorrt.sh
```
Before runing this script, please download install additionaly **opencv-contrib** and **pycuda**

   * Copy the resulting **libmyplugins.so** to the folder from which you will run this notebook
   * Replace the data model **data/models/Detector/yolov5engine/yolov5s-2021-07-28.engine** with the one you just got from running the script

Convert options and ocr
* options (inference/convertors/options2tensorrt/convert_numberplate_options_to_onnx.py)
* ocrs (inference/convertors/ocr2tensorrt/convert_ocr_to_onnx.py)

In [1]:
import os
import sys

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

current_dir = os.getcwd()
nomeroff_net_dir = os.path.join(current_dir, "../../../")
sys.path.append(nomeroff_net_dir)

In [3]:
import matplotlib.pyplot as plt
from glob import glob
import pycuda.autoinit

from nomeroff_net.tools import unzip
from nomeroff_net.pipelines.number_plate_detection_and_reading_trt_runtime import NumberPlateDetectionAndReadingTrtRuntime

In [5]:
number_plate_detection_and_reading_trt_runtime = NumberPlateDetectionAndReadingTrtRuntime(
    "number_plate_detection_and_reading_runtime", 
    image_loader="opencv", # Try 'turbo' for faster performance.
    
    # numberplate detector trt paths
    path_to_model=os.path.join(nomeroff_net_dir,
                               "./data/model_repository/yolov5s/1/model.engine"),
    plugin_lib=os.path.join(nomeroff_net_dir, 
                            "./data/model_repository/yolov5s/1/libmyplugins.so"),
    
    # numberplate classification trt paths
    path_to_classification_model=os.path.join(nomeroff_net_dir,
                                              "./data/model_repository/numberplate_options/1/model.trt"),
    options = {
        "class_region": [
                "military",
                "eu_ua_2015",
                "eu_ua_2004",
                "eu_ua_1995",
                "eu",
                "xx_transit",
                "ru",
                "kz",
                "eu-ua-fake-dpr",
                "eu-ua-fake-lpr",
                "ge",
                "by",
                "su",
                "kg",
                "am"
            ],
            "count_lines": [
                1,
                2,
                3
            ],
    },
    
    # numberplate text recognition trt paths
    prisets={
        "eu_ua_2004_2015": {
            "for_regions": ["eu_ua_2015", "eu_ua_2004"],
            "model_path": os.path.join(nomeroff_net_dir, 
                                       "./data/model_repository/ocr-eu_ua_2004_2015/1/model.trt")
        },
        "eu_ua_1995": {
            "for_regions": ["eu_ua_1995"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-eu_ua_1995/1/model.trt")
        },
        "eu": {
            "for_regions": ["eu"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-eu/1/model.trt")
        },
        "ru": {
            "for_regions": ["ru", "eu-ua-ordlo-lpr", "eu-ua-ordlo-dpr"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-ru/1/model.trt")
        },
        "kz": {
            "for_regions": ["kz"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-kz/1/model.trt")
        },
        "ge": {
            "for_regions": ["ge"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-ge/1/model.trt")
        },
        "su": {
            "for_regions": ["su"],
            "model_path": os.path.join(nomeroff_net_dir,
                                       "./data/model_repository/ocr-su/1/model.trt")
        }
    },
    default_label = "eu",
)

Loading weights from checkpoint (/var/www/nomeroff-net/examples/ju/benchmark/../../../nomeroff_net/tools/../../data/./models/np_points_craft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-net/examples/ju/benchmark/../../../nomeroff_net/tools/../../data/./models/np_points_craft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


[TensorRT] WARNING: The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.

[TensorRT] INFO: [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 3549, GPU 5907 (MiB)
[TensorRT] INFO: Loaded engine size: 24 MB
[TensorRT] INFO: [MemUsageSnapshot] deserializeCudaEngine begin: CPU 3573 MiB, GPU 5907 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 3583, GPU 5931 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuDNN: CPU +0, GPU +11, now: CPU 3583, GPU 5942 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 3583, GPU 5926 (MiB)
[TensorRT] INFO: [MemUsageSnapshot] deserializeCudaEngine end: CPU 3583 MiB, GPU 5926 

[TensorRT] INFO: [MemUsageSnapshot] deserializeCudaEngine end: CPU 3591 MiB, GPU 6200 MiB
[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation begin: CPU 3574 MiB, GPU 6200 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 3574, GPU 6208 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 3574, GPU 6216 (MiB)
[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation end: CPU 3577 MiB, GPU 6220 MiB
[TensorRT] WARNING: The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. TensorRT maintains only a single logger pointer at any given time, so the existing value, which can be retrieved with getLogger(), will be used instead. In order to use a new logger, first destroy all existing builder, runner or refitter objects.

[TensorRT] INFO: [MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 3577, GPU 6220 (MiB)
[TensorRT] WARNING: The logger passed into c

In [6]:
num_run = 1
batch_size = 1
num_workers = 1

# change on ./data/examples/benchmark_oneline_np_images/*
images = glob(os.path.join(nomeroff_net_dir, "./data/examples/benchmark_oneline_np_images/1.jpeg"))

number_plate_detection_and_reading_trt_runtime.clear_stat()

for i in range(num_run):
    print(f"pass {i}")
    outputs = number_plate_detection_and_reading_trt_runtime(
        images, 
        batch_size=batch_size,
        num_workers=num_workers)

pass 0


[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation begin: CPU 4738 MiB, GPU 6971 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 4738, GPU 6979 (MiB)
[TensorRT] INFO: [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 4738, GPU 6987 (MiB)
[TensorRT] INFO: [MemUsageSnapshot] ExecutionContext creation end: CPU 4738 MiB, GPU 6991 MiB
[TensorRT] INFO: [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 4738, GPU 6975 (MiB)


In [7]:
timer_stat = number_plate_detection_and_reading_trt_runtime.get_timer_stat(len(images)*num_run)

### Jetson Xavier Tensorrt

In [10]:
print(f"Processed {len(images)} photos")
print(f"One photo process {timer_stat['NumberPlateDetectionAndReadingTrtRuntime.call']} seconds")
print()
print(f"detect_bbox_time_all {timer_stat['NumberPlateLocalizationTrt.call']} per one photo")
print(f"craft_time_all {timer_stat['NumberPlateKeyPointsDetection.call']} per one photo")
print(f"classification_time_all {timer_stat['NumberPlateClassificationTrt.call']} per one photo")
print(f"ocr_time_all {timer_stat['NumberPlateTextReadingTrt.call']} per one photo")

Processed 1 photos
One photo process 1.5155892372131348 seconds

detect_bbox_time_all 0.023494243621826172 per one photo
craft_time_all 1.4205985069274902 per one photo
classification_time_all 0.006005048751831055 per one photo
ocr_time_all 0.0020470619201660156 per one photo


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + RTX 3090 Tensorrt in Docker